# Serialization and Deserialization of Memory Types

Langchain now supports the serialization of memory types that store memory using ChatMessageHistory. This enables the conversion of the memory type into a JSON file that can be stored locally and can be loaded back using the deserialization methods.

In [8]:
import json

from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from tests.unit_tests.llms.fake_llm import FakeLLM

## Serialization using `to_json`

In [7]:
# This example uses Conversation Buffer Memory
memory = ConversationBufferMemory()

memory.save_context({"input": "hi"}, {"output": "what's up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.load_memory_variables({})

print("Serialized Memory: ")
serialized = memory.to_json()
print(serialized)

Serialized Memory: 
{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'memory', 'buffer', 'ConversationBufferMemory'], 'kwargs': {}, 'obj': {'ai_prefix': 'AI', 'chat_memory': {'id': ['langchain', 'memory', 'chat_message_histories', 'in_memory', 'ChatMessageHistory'], 'kwargs': {}, 'lc': 1, 'obj': {'messages': [{'data': {'additional_kwargs': {}, 'content': 'hi', 'example': False, 'type': 'human'}, 'type': 'human'}, {'data': {'additional_kwargs': {}, 'content': "what's up", 'example': False, 'type': 'ai'}, 'type': 'ai'}, {'data': {'additional_kwargs': {}, 'content': 'not much you', 'example': False, 'type': 'human'}, 'type': 'human'}, {'data': {'additional_kwargs': {}, 'content': 'not much', 'example': False, 'type': 'ai'}, 'type': 'ai'}]}, 'type': 'constructor'}, 'human_prefix': 'Human', 'input_key': None, 'memory_key': 'history', 'output_key': None, 'return_messages': False}}


## Deserialization using `from_json`

In [6]:
# This example takes the JSON created above and deserializes it into a Conversation Buffer Memory object
deserialized = ConversationBufferMemory.from_json(json.dumps(serialized))

print(deserialized == memory)

True


For `ConversationSummaryMemory`, `ConversationSummaryBufferMemory`, and `ConversationEntityMemory` `from_json` requires an additional argument of an `llm`.

In [10]:
# This example uses FakeLLM, it can be replaced with any other LLM
summary_memory = ConversationSummaryMemory(llm=FakeLLM())

summary_memory.save_context({"input": "hi"}, {"output": "what is up"})
summary_memory.load_memory_variables({})

serialized_summary_memory = summary_memory.to_json()
print(serialized_summary_memory)

{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'memory', 'summary', 'ConversationSummaryMemory'], 'kwargs': {'llm': 'FakeLLM'}, 'obj': {'ai_prefix': 'AI', 'buffer': 'foo', 'chat_memory': {'id': ['langchain', 'memory', 'chat_message_histories', 'in_memory', 'ChatMessageHistory'], 'kwargs': {}, 'lc': 1, 'obj': {'messages': [{'data': {'additional_kwargs': {}, 'content': 'hi', 'example': False, 'type': 'human'}, 'type': 'human'}, {'data': {'additional_kwargs': {}, 'content': 'what is up', 'example': False, 'type': 'ai'}, 'type': 'ai'}]}, 'type': 'constructor'}, 'human_prefix': 'Human', 'input_key': None, 'llm': {'id': ['tests', 'unit_tests', 'llms', 'fake_llm', 'FakeLLM'], 'lc': 1, 'repr': 'FakeLLM()', 'type': 'not_implemented'}, 'memory_key': 'history', 'output_key': None, 'prompt': {'id': ['langchain_core', 'prompts', 'prompt', 'PromptTemplate'], 'kwargs': {'input_variables': ['new_lines', 'summary'], 'template': 'Progressively summarize the lines of conversation provided, adding o

In [10]:
llm = FakeLLM()
json_str = json.dumps(serialized_summary_memory)
revived_obj = ConversationSummaryMemory.from_json(json_str, llm=llm)
print(revived_obj == summary_memory)

True


In case the memory type in JSON uses an `llm` that does not match with the `llm` passed into the `from_json` function in the form of the JSON string, the `from_json` function will provide a warning and continue the deserialization with the newly passed `llm`.